# <center> Trabalho 05 - Introdução ao Processamento de Imagem Digital </center>

**Aluna:** Marianna de Pinho Severo <br>
**RA**: 264960 <br>
**Professor:** Hélio Pedrini

### Passo 01: Importar bibliotecas

In [1]:
%matplotlib inline
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def plotAllImages(images, labels):
    columns = 2
    rows = 3
    i = 1
    fig=plt.figure(figsize=(15, 15))
    
    for image,label in zip(images,labels):
        print(type(image[0,0]))
        fig.add_subplot(rows, columns, i)
        plt.xlabel(label)
        plt.imshow(image)
        i = i+1

    plt.subplots_adjust(bottom=0.40, top=0.75)
    plt.show()

### Passo 02: Ler imagens

In [30]:
images = {}

In [31]:
images['building1'] = cv.imread('input_images/foto1A.jpg')
images['building2'] = cv.imread('input_images/foto1B.jpg')
images['lake1'] = cv.imread('input_images/foto2A.jpg')
images['lake2'] = cv.imread('input_images/foto2B.jpg')
images['road1'] = cv.imread('input_images/foto3B.jpg')
images['road2'] = cv.imread('input_images/foto3A.jpg')
images['desertroad1'] = cv.imread('input_images/foto4B.jpg')
images['desertroad2'] = cv.imread('input_images/foto4A.jpg')
images['field1'] = cv.imread('input_images/foto5A.jpg')
images['field2'] = cv.imread('input_images/foto5B.jpg')

### Passo 03: Converter imagens para a escala de cinza

In [32]:
gray_images = {}

In [33]:
for key in images:
    gray_images[key] = cv.cvtColor(images[key], cv.COLOR_BGR2GRAY)

In [34]:
# cv.imshow('teste', gray_images['building1'])
# cv.waitKey(0)
# cv.destroyAllWindows()

### Passo 03: Encontrar pontos de interesse e descritores invariantes

In [35]:
image_keys = list(gray_images.keys())

#### Passo 3.1: ORB

In [36]:
orb_keypoints = {}
orb_descriptors = {}

In [37]:
orb = cv.ORB_create()
pos = 0
while pos < len(image_keys):
    img1 = gray_images[image_keys[pos]]
    img2 = gray_images[image_keys[pos+1]]
        
    orb_keypoints[image_keys[pos]], orb_descriptors[image_keys[pos]] = orb.detectAndCompute(img1,None)
    orb_keypoints[image_keys[pos+1]], orb_descriptors[image_keys[pos+1]] = orb.detectAndCompute(img2,None)
    
    pos+=2

#### Passo 3.2: BRIEF

In [38]:
brief_keypoints = {}
brief_descriptors = {}

In [39]:
star = cv.xfeatures2d.StarDetector_create()
brief = cv.xfeatures2d.BriefDescriptorExtractor_create()

In [40]:
pos = 0
while pos < len(image_keys):
    img1 = gray_images[image_keys[pos]]
    img2 = gray_images[image_keys[pos+1]]
    
    brief_keypoints[image_keys[pos]] = orb.detect(img1,None) #star doesn't find interest points for field1 and field2
    brief_keypoints[image_keys[pos+1]] = orb.detect(img2,None)
    
    brief_keypoints[image_keys[pos]], brief_descriptors[image_keys[pos]] = brief.compute(img1, brief_keypoints[image_keys[pos]])
    brief_keypoints[image_keys[pos+1]], brief_descriptors[image_keys[pos+1]] = brief.compute(img2, brief_keypoints[image_keys[pos+1]])
    
    pos +=2

### Passo 04: Encontrar similaridades

Essa função retorna apenas os matches cuja correspondências entre descritores for maior do que um determinado limiar. Como não sabemos a maior distância possível entre dois descritores, utilizamos a máxima distância obtida entre os matches. Então, normalizamos o vetor de distâncias por esse máximo e subtraímos de um. Assim, quanto maior for esse resultado, menor vai ser a distância entre dois descritores, com relação à maior distância obtida e, não, à maior distância possível.

In [41]:
def getCorrespondingMatches(matches, corr_level=0.5):
    if matches == []:
        return []
    
    distances = [m.distance for m in matches]
    distances = distances/np.max(distances)
    
    corr_values = np.ones((1,len(distances))) - distances
    corr_indexes = np.where(corr_values >= corr_level)[1]
    
    corresponding = [matches[idx] for idx in corr_indexes]
    
    return corresponding if len(corresponding) > 0 else matches

In [42]:
orb_matches = {}
brief_matches = {}

In [43]:
brute_force = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

In [44]:
# brute_force = cv.DescriptorMatcher_create(cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

In [45]:
pos = 0
while pos < len(image_keys):
    key1 = image_keys[pos]
    key2 = image_keys[pos+1]
    
    orb_matches[key1+ '_'+key2] = brute_force.match(orb_descriptors[key1],orb_descriptors[key2])
    orb_matches[key1+ '_'+key2] = sorted(orb_matches[key1+ '_'+key2], key = lambda x:x.distance)
    orb_matches[key1+ '_'+key2] = getCorrespondingMatches(orb_matches[key1+ '_'+key2],0.3)
    
    brief_matches[key1+ '_'+key2] = brute_force.match(brief_descriptors[key1],brief_descriptors[key2])
    brief_matches[key1+ '_'+key2] = sorted(brief_matches[key1+ '_'+key2], key = lambda x:x.distance)
    brief_matches[key1+ '_'+key2] = getCorrespondingMatches(brief_matches[key1+ '_'+key2])
    
    pos+=2

In [46]:
# # Draw first 10 matches.
# plt.figure(figsize=(10,10))
# imMatches = cv.drawMatches(gray_images['desert_road1'], orb_keypoints['desert_road1'],gray_images['desert_road2'], orb_keypoints['desert_road2'], orb_matches['desert_road1_desert_road2'], None)
# plt.imshow(imMatches),plt.show()

### Passo 05: Estimar matriz de homografia

In [47]:
orb_homography_matrix = {}
brief_homography_matrix = {}

In [48]:
for key in orb_matches:
    points1 = np.zeros((len(orb_matches[key]), 2), dtype=np.float32)
    points2 = np.zeros((len(orb_matches[key]), 2), dtype=np.float32)
    
    if len(orb_matches[key])>=4:
        for i, match in enumerate(orb_matches[key]):
            subkey1,subkey2 = key.split('_')

            points1[i, :] = (orb_keypoints[subkey1])[match.queryIdx].pt
            points2[i, :] = (orb_keypoints[subkey2])[match.trainIdx].pt

        orb_homography_matrix[key], mask = cv.findHomography(points1, points2, cv.RANSAC)
    

In [49]:
for key in brief_matches:
    points1 = np.zeros((len(brief_matches[key]), 2), dtype=np.float32)
    points2 = np.zeros((len(brief_matches[key]), 2), dtype=np.float32)
    
    if len(brief_matches[key])>=4:
        for i, match in enumerate(brief_matches[key]):
            subkey1,subkey2 = key.split('_')

            points1[i, :] = (brief_keypoints[subkey1])[match.queryIdx].pt
            points2[i, :] = (brief_keypoints[subkey2])[match.trainIdx].pt

        brief_homography_matrix[key], mask = cv.findHomography(points1, points2, cv.RANSAC,5.0)
    

In [50]:
for key in brief_homography_matrix:
    print("key:{}\n{}".format(key, brief_homography_matrix[key]))

key:building1_building2
[[ 8.10030703e-01  1.10831200e-01  4.28237400e+02]
 [-1.34127332e-01  9.93500367e-01  5.75715582e+01]
 [-2.09447284e-04  5.15296111e-05  1.00000000e+00]]
key:lake1_lake2
[[ 1.00175598e+00  5.90324781e-05  4.62964699e+02]
 [-6.05475729e-05  1.00122197e+00 -8.71878025e-02]
 [ 1.99085720e-06  1.05295372e-06  1.00000000e+00]]
key:road1_road2
[[ 1.00016778e+00  2.13712076e-02 -2.99192385e+01]
 [-2.01675244e-02  1.01476959e+00  1.32684758e+02]
 [-1.51295903e-05  4.72223175e-05  1.00000000e+00]]
key:desertroad1_desertroad2
[[-4.71491007e-01 -2.64025257e-01  1.83256439e+02]
 [-6.01186760e-01 -4.05202167e-01  2.52128383e+02]
 [-2.39345797e-03 -1.60457853e-03  1.00000000e+00]]
key:field1_field2
[[ 1.66804304e+00 -1.60574596e-01 -3.81870061e+01]
 [ 4.15689801e-01  1.08436980e+00  5.06288649e+01]
 [ 8.88913588e-04 -1.08570049e-04  1.00000000e+00]]


In [51]:
for key in orb_homography_matrix:
    print("key:{}\n{}".format(key, orb_homography_matrix[key]))

key:building1_building2
[[ 8.17948533e-01  1.24347475e-01  4.24088834e+02]
 [-1.33492861e-01  1.00166942e+00  5.65667993e+01]
 [-2.06849224e-04  6.27661732e-05  1.00000000e+00]]
key:lake1_lake2
[[ 9.94820349e-01  4.86939747e-04  4.63132833e+02]
 [-5.75531434e-04  9.99917525e-01  2.98149574e-02]
 [-5.69777271e-06  1.37462067e-06  1.00000000e+00]]
key:road1_road2
[[ 1.00611333e+00  2.55983645e-02 -3.09871534e+01]
 [-1.80572177e-02  1.02484832e+00  1.31902495e+02]
 [-6.36002536e-06  6.03680923e-05  1.00000000e+00]]
key:desertroad1_desertroad2
[[ 4.31838750e-01 -4.37816978e-01  1.94874440e+02]
 [ 4.14354519e-01  4.34967359e-01  2.46995734e+00]
 [-4.04824451e-05 -5.16501417e-05  1.00000000e+00]]
key:field1_field2
[[ 9.56485606e-01 -7.39491441e-02  4.42703442e+01]
 [ 4.90152164e-02  9.58158146e-01  9.53033313e+01]
 [-3.02035118e-05 -6.02380853e-05  1.00000000e+00]]


### Passo 06: Alinhar imagens

In [52]:
orb_panoramic_images = {}
brief_panoramic_images = {}

In [53]:
for key in orb_homography_matrix:
    subkey1,subkey2 = key.split('_')
    
    img1 = images[subkey1]
    img2 = images[subkey2]
    
    height, width,deep = img2.shape
    orb_panoramic_images[key] = cv.warpPerspective(img1, orb_homography_matrix[key], (width+img1.shape[1], height+img1.shape[0]))
    (orb_panoramic_images[key])[:img2.shape[0],:img2.shape[1]] = img2
    
    brief_panoramic_images[key] = cv.warpPerspective(img1, brief_homography_matrix[key], (width+img1.shape[1], height+img1.shape[0]))
    (brief_panoramic_images[key])[:img2.shape[0],:img2.shape[1]] = img2

In [54]:
# for key in brief_panoramic_images:
#     plt.imshow(brief_panoramic_images[key],)
#     plt.show()

### Passo 07: Salvar imagens

In [55]:
for key in orb_panoramic_images:
    cv.imwrite('output_images/orb_'+key+'.png', orb_panoramic_images[key])

In [27]:
for key in brief_panoramic_images:
    cv.imwrite('output_images/brief_'+key+'.png', brief_panoramic_images[key])